In [1]:
# Imports
import mtcnn
import cv2
import numpy as np

def get_boxes_for_file(path):
    '''Returns two lists: a list of the names of the image files and a list of the bounding boxes'''
    #Open file
    f = open(path,'r') 
    file = f.readlines()
    boxes = {}
    files = []
    for idx, line in enumerate(file): 
        try:
            total = int(line[:-1])
            count = 0
            filename = file[idx-1][:-1]
            boxes[filename] = []
            files.append(filename)
            while count < total:
                bb = file[idx+count+1]

                #Convert into array of integer coordinates
                num = bb.split(' ')
                num = num[:4]
                num = [int(i) for i in num]
                
                #Save into array
                boxes[filename].append(num)
                count = count+1
        except:
            # this will fail if the int conversion doesnt work.
            pass
    return boxes, files

In [2]:
# IoU code from https://www.pyimagesearch.com/2016/11/07/intersection-over-union-iou-for-object-detection/
# edited for use in our system.
def bb_intersection_over_union(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[0] + boxA[2], boxB[0] + boxB[2])
	yB = min(boxA[1] + boxB[3], boxB[1] + boxB[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = (boxA[2] + 1) * (boxA[3] + 1)
	boxBArea = (boxB[2] + 1) * (boxB[3] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)
	# return the intersection over union value
	return iou

In [5]:

boxes, files = get_boxes_for_file('./wider_face_split/wider_face_val_bbx_gt.txt')

mAP = 0 # mean accuracy precision
f1_mean = 0 # precision/recall metric

pr_curve = []

detector = mtcnn.MTCNN( './data/mtcnn_weights.npy' )

for idx, filename in enumerate(files):
    # Read in test image
    image = cv2.imread( "./WIDER_val/" + filename )
    rgbimg = cv2.cvtColor( image, cv2.COLOR_BGR2RGB )

    # Detect faces in the image
    faces = detector.detect_faces( rgbimg )

    real_boxes = boxes[filename]

    faces.sort(key=lambda f: f['box'][0])
    real_boxes.sort(key=lambda b: b[0])

    tp = 0
    fn = 0
    fp = 0

    face_idx = 0

    for box in real_boxes:
        if face_idx < len(faces):
            face_box = faces[face_idx]['box']
            if abs(box[0] - face_box[0]) <= 10:
                iou = bb_intersection_over_union(box, face_box)
                if iou >= .5:
                    tp += 1
                else:
                    fp += 1
            else:
                fn += 1
            
            if face_box[0] <= box[0]:
                face_idx += 1

        else:
            fn += 1

    recall = np.nan_to_num(np.divide(tp, (tp + fn)))
    precision = np.nan_to_num(np.divide(tp, (tp + fp)))

    # print(recall, precision)

    pr_curve.append((recall, precision))

print(f1_mean)

nan


In [7]:
f1_mean = 0

for pr in pr_curve:
    f1_mean += np.divide(2 * precision * recall, precision + recall) / len(files)

print(f1_mean)

0.49999999999999994


Basically 50% f1. its a general rating that it tends to get things right about 50% of the time

In [9]:
# I was thinking we could try calculating the mAP. mean average precision, but the calculation is a bit confusing.
# Will try looking into it later.
    

1.0), (0.4444444444444444, 1.0), (0.4444444444444444, 1.0), (0.4444444444444444, 1.0), (0.4444444444444444, 0.8), (0.45, 1.0), (0.45, 0.627906976744186), (0.45454545454545453, 1.0), (0.45454545454545453, 1.0), (0.45454545454545453, 0.8333333333333334), (0.45454545454545453, 0.8333333333333334), (0.45454545454545453, 1.0), (0.45454545454545453, 1.0), (0.45714285714285713, 0.7619047619047619), (0.4583333333333333, 0.6875), (0.46153846153846156, 1.0), (0.46153846153846156, 1.0), (0.46153846153846156, 0.75), (0.46153846153846156, 1.0), (0.4626865671641791, 0.29523809523809524), (0.4666666666666667, 0.7777777777777778), (0.4666666666666667, 1.0), (0.4666666666666667, 0.7777777777777778), (0.47058823529411764, 0.8888888888888888), (0.47058823529411764, 0.8), (0.47058823529411764, 0.8), (0.4722222222222222, 0.4358974358974359), (0.475, 0.475), (0.47619047619047616, 0.6060606060606061), (0.47619047619047616, 0.9090909090909091), (0.47619047619047616, 0.7692307692307693), (0.4782608695652174, 0